In [132]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, Session
from sqlalchemy import select, delete, update
from sqlalchemy import Column, Integer, String, ForeignKey, Boolean, DateTime, func
from sqlalchemy.orm import relationship

In [133]:
engine = create_engine('postgresql:///euphoria', echo=True, future=True)
# engine = create_engine('sqlite:///:memory:', echo=False, future=True)
session = Session(engine, future=True)
Base = declarative_base()

In [134]:
class Box(Base):
    __tablename__ = 'boxes'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    size = Column(String, nullable=False)
    essential = Column(Boolean)
    warm = Column(Boolean)
    liquid = Column(Boolean)
    items = relationship('Item', back_populates='box', cascade='all, delete')

    def __repr__(self):
        return f'Box(name={self.name!r}, size={self.size!r}, essential={self.essential!r}, warm={self.warm!r}, liquid={self.liquid!r})'


class Item(Base):
    __tablename__ = 'items'
    id = Column(Integer, primary_key=True)
    box_id = Column(Integer, ForeignKey('boxes.id'), nullable=False)
    name = Column(String, nullable=False)
    essential = Column(Boolean, nullable=False)
    warm = Column(Boolean, nullable=False)
    liquid = Column(Boolean, nullable=False)
    box = relationship('Box', back_populates='items')

    def __repr__(self):
        return f'Item(box_id={self.box_id!r}, name={self.name!r}, essential={self.essential!r}, warm={self.warm!r}, liquid={self.liquid!r})'

In [135]:
Base.metadata.create_all(engine)

2022-05-13 17:16:00,200 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-05-13 17:16:00,202 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-05-13 17:16:00,204 INFO sqlalchemy.engine.Engine select current_schema()
2022-05-13 17:16:00,205 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-05-13 17:16:00,206 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-05-13 17:16:00,206 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-05-13 17:16:00,207 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-13 17:16:00,209 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-05-13 17:16:00,210 INFO sqlalchemy.engine.Engine [generated in 0.00103s] {'name': 'boxes'}
2022-05-13 17:16:00,217 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name

In [136]:
from euphoria.inventory.test_boxes import box_list, item_list

In [137]:
with session:
    for box in box_list:
        session.add(Box(**box))
    for item in item_list:
        session.add(Item(**item))
    session.commit()

2022-05-13 17:16:39,276 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-13 17:16:39,280 INFO sqlalchemy.engine.Engine INSERT INTO boxes (name, size, essential, warm, liquid) VALUES (%(name)s, %(size)s, %(essential)s, %(warm)s, %(liquid)s) RETURNING boxes.id
2022-05-13 17:16:39,281 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ({'name': 'Computers', 'size': 'medium', 'essential': False, 'warm': True, 'liquid': False}, {'name': 'Monitors and Clock', 'size': 'xlarge', 'essential': False, 'warm': True, 'liquid': False}, {'name': 'Food Staples', 'size': 'medium', 'essential': True, 'warm': True, 'liquid': True}, {'name': 'The Box with the Longest Name Ever in the World', 'size': 'medium', 'essential': False, 'warm': False, 'liquid': False}, {'name': 'D', 'size': 'medium', 'essential': True, 'warm': False, 'liquid': True}, {'name': 'K Cups', 'size': 'medium', 'essential': False, 'warm': True, 'liquid': False}, {'name': 'TV', 'size': 'xlarge', 'essential': False, 'warm': True, 

In [138]:
with session:
    boxes = session.query(Box).all()
    print(boxes)

2022-05-13 17:16:42,837 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-13 17:16:42,838 INFO sqlalchemy.engine.Engine SELECT boxes.id AS boxes_id, boxes.name AS boxes_name, boxes.size AS boxes_size, boxes.essential AS boxes_essential, boxes.warm AS boxes_warm, boxes.liquid AS boxes_liquid 
FROM boxes
2022-05-13 17:16:42,839 INFO sqlalchemy.engine.Engine [generated in 0.00066s] {}
[Box(name='Computers', size='medium', essential=False, warm=True, liquid=False), Box(name='Monitors and Clock', size='xlarge', essential=False, warm=True, liquid=False), Box(name='Food Staples', size='medium', essential=True, warm=True, liquid=True), Box(name='The Box with the Longest Name Ever in the World', size='medium', essential=False, warm=False, liquid=False), Box(name='D', size='medium', essential=True, warm=False, liquid=True), Box(name='K Cups', size='medium', essential=False, warm=True, liquid=False), Box(name='TV', size='xlarge', essential=False, warm=True, liquid=False), Box(name='Widescree

In [121]:
with session:
    box = session.get(Box, 5)
    print(box)
    print()
    print(box.items)

Box(name='D', size='medium', essential=True, warm=False, liquid=True)

[Item(box_id=5, name='S', essential=True, warm=True, liquid=True), Item(box_id=5, name='cords', essential=True, warm=False, liquid=False), Item(box_id=5, name='lamp', essential=True, warm=False, liquid=False), Item(box_id=5, name='S', essential=True, warm=True, liquid=True), Item(box_id=5, name='Do you want some more?', essential=True, warm=False, liquid=False), Item(box_id=5, name='Some cords that have a long name that messes it up', essential=False, warm=False, liquid=True)]


In [122]:
with session:
    box = session.get(Box, 5)
    box.name = 'awesome box'
    session.commit()

In [123]:
with session:
    box = session.get(Box, 5)
    print(box)

Box(name='awesome box', size='medium', essential=True, warm=False, liquid=True)


In [124]:
with session:
    items = session.execute(select(Item).where(Item.box_id == 5)).scalars().all()
    print(items)

[Item(box_id=5, name='S', essential=True, warm=True, liquid=True), Item(box_id=5, name='cords', essential=True, warm=False, liquid=False), Item(box_id=5, name='lamp', essential=True, warm=False, liquid=False), Item(box_id=5, name='S', essential=True, warm=True, liquid=True), Item(box_id=5, name='Do you want some more?', essential=True, warm=False, liquid=False), Item(box_id=5, name='Some cords that have a long name that messes it up', essential=False, warm=False, liquid=True)]


In [125]:
with session:
    box = session.get(Box, 5)
    session.delete(box)
    session.commit()

In [126]:
with session:
    box = session.get(Box, 5)
    print(box)
    print()
    print(box.items if box else 'None')

None

None


In [127]:
with session:
    items = session.execute(select(Item).where(Item.box_id == 5)).scalars().all()
    print(items)

[]


In [128]:
with session:
    count = session.query(Box).count()
    print(count)

14


In [129]:
with session:
    count = session.execute(select(func.count()).select_from(Box)).scalar()
    print(count)

14


## Association Tables - Many to Many

In [59]:
from sqlalchemy import Table

In [60]:
# association_table = Table('association', Base.metadata,
#     Column('left_id', Integer, ForeignKey('left.id')),
#     Column('right_id', Integer, ForeignKey('right.id'))
# )

# class Parent(Base):
#     __tablename__ = 'left'
#     id = Column(Integer, primary_key=True)
#     children = relationship(
#         "Child",
#         secondary=association_table,
#         back_populates="parents",
#         cascade="all, delete"
#     )

# class Child(Base):
#     __tablename__ = 'right'
#     id = Column(Integer, primary_key=True)
#     parents = relationship(
#         "Parent",
#         secondary=association_table,
#         back_populates="children",
#     )

In [61]:
class Apartment(Base):
    __tablename__ = 'apartments'
    id = Column(Integer, primary_key=True)
    features = relationship(
        "Feature",
        back_populates="apartments",
        cascade="all, delete"
    )

class Feature(Base):
    __tablename__ = 'features'
    id = Column(Integer, primary_key=True)
    apt_id = Column(Integer, ForeignKey('apartments.id'))
    apartment = relationship(
        "Apartment",
        back_populates="features",
    )

## Default Values

In [62]:
class MyModel(Base):
    __tablename__ = 'my_table'

    id = Column(Integer, primary_key=True)
    timestamp = Column(DateTime(), server_default=func.now())



In [7]:
from datetime import date, timedelta
import calendar
today = date.today()
tomorrow = today + timedelta(days=1)
yesterday = today - timedelta(days=1)
previous_7 = today - timedelta(days=7)
previous_30 = today - timedelta(days=30)
_month_days = calendar.monthrange(today.year, today.month)[1]
week_number = today.isocalendar().week
week_start = date.fromisocalendar(today.year, week_number, 1)
week_end = week_start + timedelta(days=7)
this_month = date(today.year, today.month, 1)
next_month = this_month + timedelta(days=_month_days)
this_year = date(today.year, 1, 1)
next_year = date(today.year + 1, 1, 1)

In [8]:
print(f'{today = }')
print(f'{tomorrow = }')
print(f'{yesterday = }')
print(f'{previous_7 = }')
print(f'{previous_30 = }')
print(f'{_month_days = }')
print(f'{week_number = }')
print(f'{week_start = }')
print(f'{week_end = }')
print(f'{this_month = }')
print(f'{next_month = }')
print(f'{this_year = }')
print(f'{next_year = }')

today = datetime.date(2022, 5, 27)
tomorrow = datetime.date(2022, 5, 28)
yesterday = datetime.date(2022, 5, 26)
previous_7 = datetime.date(2022, 5, 20)
previous_30 = datetime.date(2022, 4, 27)
_month_days = 31
week_number = 21
week_start = datetime.date(2022, 5, 23)
week_end = datetime.date(2022, 5, 30)
this_month = datetime.date(2022, 5, 1)
next_month = datetime.date(2022, 6, 1)
this_year = datetime.date(2022, 1, 1)
next_year = datetime.date(2023, 1, 1)


## Basic Association Example

In [ ]:
"""Illustrate a many-to-many relationship between an
"Order" and a collection of "Item" objects, associating a purchase price
with each via an association object called "OrderItem"

The association object pattern is a form of many-to-many which
associates additional data with each association between parent/child.

The example illustrates an "order", referencing a collection
of "items", with a particular price paid associated with each "item".

"""

from datetime import datetime

from sqlalchemy import and_
from sqlalchemy import Column
from sqlalchemy import create_engine
from sqlalchemy import DateTime
from sqlalchemy import Float
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.orm import Session


Base = declarative_base()


class Order(Base):
    __tablename__ = "order"

    order_id = Column(Integer, primary_key=True)
    customer_name = Column(String(30), nullable=False)
    order_date = Column(DateTime, nullable=False, default=datetime.now())
    order_items = relationship(
        "OrderItem", cascade="all, delete-orphan", backref="order"
    )

    def __init__(self, customer_name):
        self.customer_name = customer_name


class Item(Base):
    __tablename__ = "item"
    item_id = Column(Integer, primary_key=True)
    description = Column(String(30), nullable=False)
    price = Column(Float, nullable=False)

    def __init__(self, description, price):
        self.description = description
        self.price = price

    def __repr__(self):
        return "Item(%r, %r)" % (self.description, self.price)


class OrderItem(Base):
    __tablename__ = "orderitem"
    order_id = Column(Integer, ForeignKey("order.order_id"), primary_key=True)
    item_id = Column(Integer, ForeignKey("item.item_id"), primary_key=True)
    price = Column(Float, nullable=False)

    def __init__(self, item, price=None):
        self.item = item
        self.price = price or item.price

    item = relationship(Item, lazy="joined")


if __name__ == "__main__":
    engine = create_engine("sqlite://")
    Base.metadata.create_all(engine)

    session = Session(engine)

    # create catalog
    tshirt, mug, hat, crowbar = (
        Item("SA T-Shirt", 10.99),
        Item("SA Mug", 6.50),
        Item("SA Hat", 8.99),
        Item("MySQL Crowbar", 16.99),
    )
    session.add_all([tshirt, mug, hat, crowbar])
    session.commit()

    # create an order
    order = Order("john smith")

    # add three OrderItem associations to the Order and save
    order.order_items.append(OrderItem(mug))
    order.order_items.append(OrderItem(crowbar, 10.99))
    order.order_items.append(OrderItem(hat))
    session.add(order)
    session.commit()

    # query the order, print items
    order = session.query(Order).filter_by(customer_name="john smith").one()
    print(
        [
            (order_item.item.description, order_item.price)
            for order_item in order.order_items
        ]
    )

    # print customers who bought 'MySQL Crowbar' on sale
    q = session.query(Order).join("order_items", "item")
    q = q.filter(
        and_(Item.description == "MySQL Crowbar", Item.price > OrderItem.price)
    )

    print([order.customer_name for order in q])